In [1]:
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
#loading the data
train_data = pd.read_csv("dataset/train.csv")
test_data = pd.read_csv("dataset/test.csv")

In [3]:
train_data.head()

,Image,Class
0,image7042.jpg,Food
1,image3327.jpg,misc
2,image10335.jpg,Attire
3,image8019.jpg,Food
4,image2128.jpg,Attire


In [4]:
#Encoding the categorical values to numerical values
le = LabelEncoder()
train_data['Class']=le.fit_transform(train_data['Class'])
# train_data['Class']

In [5]:
#Separating the x_train and y_train
x_train=[]
y_train=[]
i=0
for img in train_data['Image']:
    path = os.path.join("dataset/Train Images",img)
    train_img = cv2.imread(path)
    train_img=cv2.resize(train_img,(150,150))
    train_img = train_img.astype('float32')
    x_train.append(train_img)
    y_train.append(train_data['Class'][i])
    i=i+1
# print(x_train[0])


In [6]:
x_test=[]
for img in test_data['Image']:
    path = os.path.join("dataset/Test Images",img)
    test_img = cv2.imread(path)
    test_img=cv2.resize(test_img,(150,150))
    test_img = test_img.astype('float32')
    x_test.append(test_img)

In [7]:
x_train=np.array(x_train)
y_train=np.array(y_train)
x_test=np.array(x_test)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)

(5983, 150, 150, 3)
(5983,)
(3219, 150, 150, 3)


In [ ]:
import keras
from keras.models import Sequential
from keras.applications.resnet50 import ResNet50
from keras.layers import Dense, Flatten, Conv2D, Dropout, Activation, MaxPooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import RMSprop, Adam
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

Using TensorFlow backend.


In [ ]:
num_classes = 4
batch_size = 128
epochs = 20

In [ ]:
#creating the model
model = Sequential()
model.add(ResNet50(include_top=False, pooling='avg', weights='imagenet', input_shape=(150,150,3)))
model.add(Dense(num_classes, activation = "softmax"))
model.layers[0].trainable = False
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 8196      
Total params: 23,595,908
Trainable params: 8,196
Non-trainable params: 23,587,712
_________________________________________________________________


In [ ]:
#Data Augmentation
datagen = ImageDataGenerator(
      featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

datagen.fit(x_train)

In [ ]:
#compiling the model
optimiser = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(loss='categorical_crossentropy', optimizer=optimiser, metrics=['accuracy'])

In [ ]:
# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='loss', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [ ]:
model.fit_generator(datagen.flow(x_train,to_categorical(y_train,4), batch_size=batch_size),
                              epochs = epochs,                          
                              callbacks=[learning_rate_reduction], workers=4)
# model.fit(x_train, to_categorical(y_train,4), batch_size=batch_size, epochs=epochs, callbacks=[learning_rate_reduction])

Epoch 1/20
47/47 [==============================] - 261s 6s/step - loss: 0.9790 - accuracy: 0.6092
Epoch 2/20
47/47 [==============================] - 245s 5s/step - loss: 0.6911 - accuracy: 0.7414
Epoch 3/20
47/47 [==============================] - 245s 5s/step - loss: 0.6399 - accuracy: 0.7571
Epoch 4/20
47/47 [==============================] - 245s 5s/step - loss: 0.5920 - accuracy: 0.7784
Epoch 5/20
47/47 [==============================] - 244s 5s/step - loss: 0.5678 - accuracy: 0.7852
Epoch 6/20
47/47 [==============================] - 244s 5s/step - loss: 0.5493 - accuracy: 0.7939
Epoch 7/20
47/47 [==============================] - 245s 5s/step - loss: 0.5360 - accuracy: 0.7978
Epoch 8/20
47/47 [==============================] - 245s 5s/step - loss: 0.5316 - accuracy: 0.7988
Epoch 9/20
47/47 [==============================] - 244s 5s/step - loss: 0.5082 - accuracy: 0.8081
Epoch 10/20
47/47 [==============================] - 244s 5s/step - loss: 0.4897 - accuracy: 0.8135
Epoch 11/

In [ ]:
y_test = model.predict(x_test)
label = [np.argmax(i) for i in y_test]
y_test = le.inverse_transform(label)
print(y_test)
submission = pd.DataFrame({'Image':test_data.Image, 'Class':y_test})
submission.to_csv('submission.csv',index=False)
